# Esercitazione Machine Learning - AutomaParts S.p.A.

In questo notebook andremo a sviluppare un modello di machine learning per l'azienda **AutomaParts S.p.A.**.
L'obiettivo è prevedere se un pezzo prodotto è difettoso (defect_label = 1) o conforme (defect_label = 0) basandoci su varie misure rilevate durante la produzione.

## 1. Importazione delle librerie
Iniziamo importando le librerie necessarie per l'analisi e la modellazione.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Librerie per il machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

%matplotlib inline

## 2. Caricamento e Analisi dei Dati
Carichiamo il dataset `parts_production_data.csv` e diamo un'occhiata alle prime righe per capire come è fatto.

In [ ]:
df = pd.read_csv('parts_production_data.csv')
df.head()

In [ ]:
print("Dimensione del dataset:", df.shape)
df.info()

Vediamo un po' di statistiche descrittive per le colonne numeriche.

In [ ]:
df.describe()

### Analisi della variabile target
Controlliamo quanti pezzi sono difettosi e quanti no. È importante vedere se le classi sono bilanciate.

In [ ]:
print(df['defect_label'].value_counts())
sns.countplot(x='defect_label', data=df)
plt.title('Distribuzione Difetti')
plt.show()

Come possiamo vedere, c'è uno sbilanciamento (ci sono meno difetti che pezzi buoni), il che è normale.

### Matrice di correlazione
Vediamo se ci sono variabili molto correlate tra loro o con il target.

In [ ]:
plt.figure(figsize=(12,8))
# Selezioniamo solo le colonne numeriche per la correlazione
numeric_df = df.select_dtypes(include=[np.number])
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matrice di Correlazione')
plt.show()

## 3. Pulizia e Preparazione dei Dati (Data Cleaning)

Adesso controlliamo se ci sono valori mancanti (NaN).

In [ ]:
df.isnull().sum()

Se ci sono valori mancanti, li riempiamo con la media della colonna. In questo modo non perdiamo i dati.

In [ ]:
# Riempio i valori nulli con la media solo per le colonne numeriche
colonne_numeriche = df.select_dtypes(include=[np.number]).columns
df[colonne_numeriche] = df[colonne_numeriche].fillna(df[colonne_numeriche].mean())

# Ricontrollo
df.isnull().sum().sum()

Eliminiamo le colonne che non servono per la predizione, come `part_id` (è solo un codice) e `production_timestamp` (per ora non facciamo analisi temporali complesse).

In [ ]:
df = df.drop(columns=['part_id', 'production_timestamp'], errors='ignore')
df.head()

### Gestione Variabili Categoriche
Abbiamo colonne come `line_id`, `station_id` ecc. che sono categorie. Le trasformiamo in numeri usando il `LabelEncoder` o `get_dummies`. Qui uso `get_dummies` per semplicità sulle variabili con poche categorie, e `LabelEncoder` per i batch se sono troppi, ma facciamo tutto con `get_dummies` per fare prima, oppure LabelEncoder se sono tante uniche.

In [ ]:
# Vediamo quante categorie uniche ci sono
print("Batch unici:", df['material_batch'].nunique())
print("Line ID unici:", df['line_id'].nunique())

Visto che `material_batch` ha molte varianti, usiamo LabelEncoder per quella, e get_dummies per le altre (che però in questo dataset sono già numeriche o quasi, `line_id` è numerico ma rappresenta una categoria).

In [ ]:
# Converto measure e altre se necessario. Ma le colonne ID sembrano numeri interi, le trattiamo come numeri o categorie?
# Per semplicità le lascio come numeri per ora, ma material_batch è stringa.

le = LabelEncoder()
df['material_batch'] = le.fit_transform(df['material_batch'])

### Scaling delle Features
Le variabili hanno scale diverse (es. diametro in mm, temperatura in gradi). Meglio portarle tutte sulla stessa scala.

In [ ]:
# Divido in X e y
X = df.drop('defect_label', axis=1)
y = df['defect_label']

# Divido in Training e Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set:", X_train.shape)
print("Test set:", X_test.shape)

In [ ]:
scaler = StandardScaler()

# Adatto lo scaler solo sul train per evitare data leakage
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## 4. Modellazione
Proveremo tre modelli diversi come richiesto:
1. Logistic Regression
2. Decision Tree
3. Random Forest

### Modello 1: Logistic Regression

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train_scaled, y_train)

y_pred_log = log_reg.predict(X_test_scaled)

print("Accuracy Logistic Regression:", accuracy_score(y_test, y_pred_log))

### Modello 2: Decision Tree
Gli alberi decisionali sono facili da interpretare.

In [ ]:
# Non serve scaling per i tree based solitamente, ma usiamo X_train normale
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)

y_pred_tree = tree_clf.predict(X_test)

print("Accuracy Decision Tree:", accuracy_score(y_test, y_pred_tree))

### Modello 3: Random Forest
Un insieme di alberi, di solito più robusto.

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)

y_pred_rf = rf_clf.predict(X_test)

print("Accuracy Random Forest:", accuracy_score(y_test, y_pred_rf))

## 5. Valutazione e Confronto

Confrontiamo i risultati dei tre modelli usando metriche più dettagliate come la Confusion Matrix e il Classification Report.

In [ ]:
print("--- Logistic Regression ---")
print(classification_report(y_test, y_pred_log))

print("\n--- Decision Tree ---")
print(classification_report(y_test, y_pred_tree))

print("\n--- Random Forest ---")
print(classification_report(y_test, y_pred_rf))

Andiamo a vedere la matrice di confusione per il modello migliore (probabilmente il Random Forest).

In [ ]:
cm = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predetto')
plt.ylabel('Reale')
plt.title('Confusion Matrix Random Forest')
plt.show()

### Feature Importance
Vediamo quali variabili hanno influito di più sulla decisione del modello.

In [ ]:
importances = rf_clf.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,6))
plt.title("Feature Importance (Random Forest)")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
plt.show()

## Conclusioni

Dai risultati ottenuti vediamo che il **Random Forest** (o il modello che ha performato meglio) è il più affidabile.
L'analisi ha mostrato che alcune variabili come il diametro e la temperatura sono molto importanti per predire i difetti.

Per AutomaParts S.p.A., utilizzare questo modello in produzione potrebbe permettere di intercettare molti pezzi difettosi prima che arrivino al cliente, riducendo i costi.
Bisogna però fare attenzione ai "Falsi Negativi" (pezzi difettosi predetti come buoni), perché quelli sono i più pericolosi. Magari si potrebbe abbassare la soglia di probabilità per essere più cautelativi.